### Solutions to the problems of Chapter 8 
## Trees 

##### Implementations to the Tree ADT, Binary Tree, Linked Binary Tree, Array Binary Tree

In [18]:
class Tree:
    class Position: 
        def element(self): 
            raise NotImplementedError('must be implemented by subclass') 
        def __eq__(self, other):
            raise NotImplementedError('must be implemented by subclass') 
        def __ne__(self, other):
            return not (self == other)
    
    ## accessor methods
    
    def root(self):
        raise NotImplementedError('must be implemented by subclass')
    def parent(self, p):
        raise NotImplementedError('must be implemented by subclass') 
    def is_root(self, p):
        return self.root() == p 
    def num_children(self, p):
        raise NotImplementedError('must be implemented by subclass')
    def children(self, p):
        raise NotImplementedError('must be implemented by subclass')
    def is_leaf(self, p):
        return self.num_children(p) == 0
    def __len__(self):
        raise NotImplementedError('must be implemented by subclass') 
    def is_empty(self):
        return len(self) == 0
    def depth(self, p):
        if self.is_root(p):
            return 0
        else:
            return 1 + self.depth(self.parent(p)) 
        
    def height(self, p = None):
        def _height(p): 
            if self.is_leaf(p):
                return 0
            else:
                return 1 + max(_height(n) for n in self.children(p))
            
        if p is None:
            p = self.root()
        return _height(p)
    
        
class BinaryTree(Tree): 
    def left(self, p):
        raise NotImplementedError('must be implemented by subclass')
    def right(self, p):
        raise NotImplementedError('must be implemented by subclass') 
        
    def sibling(self, p):
        parent = self.parent(p)
        if parent is None:
            return None
        else:
            if p == self.left(parent):
                return self.right(parent)
            else:
                return self.left(parent)
            
    def children(self, p):
        if self.left(p) is not None:
            yield self.left(p)
        if self.right(p) is not None:
            yield self.right(p) 

class LinkedBinaryTree(BinaryTree):
    class _Node:
        __slots__ = '_element', '_parent', '_left', '_right'
        def init (self, element, parent=None, left=None, right=None):
            self._element = element
            self._parent = parent 
            self._left = left
            self._right = right
    
    class Position(BinaryTree.Position):
        def __init__(self, container, node):
            self._container = container 
            self._node = node
        def element(self):
            return self._node._element 
        def __eq__(self, other): 
            return type(other) is type(self) and self._node is other._node      
        
    def _validate(self, p):
        if not isinstance(p, self.Position):
            raise TypeError("p must be proper Position type")
        if p._container is not self:
            raise ValueError('p does not belong to this container')
        if p._node._parent is p._node:
            raise ValueError('p is no longer valid')
        return p._node  
    
    def _make_position(self, node): 
        return self.Position(self, node) if node is not None else None 
    
    
    def __init__(self):
        self._root = None
        self._size = 0
        
    def __len__(self):
        return self._size 
    
    def root(self):
        return self._make_position(self._root)
    
    def parent(self, p):
        return self._make_position(self._validate(p)._parent)
    
    def left(self, p):
        return self._make_position(self._validate(p)._left)
    
    def right(self, p):
        return self._make_position(self._validate(p)._right)
    
    def num_children(self, p):
        node = self._validate(p)
        count = 0
        if node._left is not None:
            count+=1 
        if node._right is not None:
            count+1 
        return count 
    
    def add_root(self, e):
        if self._root is not None: raise ValueError('root already exists !')
        self._root = self._Node(e)
        self._size = 1
        return self._make_position(self._root)
    
    def add_left(self, p, e):
        positioned_node = self._validate(p)
        if positioned_node._left is not None: raise ValueError('left position is not empty !')
        positioned_node._left = self._Node(e, parent = positioned_node)
        self._size += 1 
        return self._make_position(positioned_node._left)
    
    def add_right(self, p, e):
        positioned_node = self._validate(p)
        if positioned_node._right is not None: raise ValueError('left position is not empty !')
        positioned_node._right = self._Node(e, parent = positioned_node)
        self._size += 1
        return self._make_position(positioned_node._right) 
    
    def _delete(self, p):
        node = self._validate(p)
        if self.num_children(p)==2: raise ValueError('p has 2 children')
        child = node._left if node._left else node._right
        if child is not None:
            child._parent = node._parent 
        if node is self._root:
            self._root = child 
        else:
            parent = node._parent
            if child == node._left:
                parent._left = child
            else:
                parent._right = child
        size -= 1
        node._parent = node
        return node._element 
    
    def _attach(self, p, t1, t2):
        node = self._validate(p)
        if not self.is_leaf(p): raise ValueError('p must be a leaf position')
        if type(self) is type(t1) is type(t2): raise TypeError('Invalid types for trees')
        self._size += len(t1) + len(t2)
        if not t1.is_empty():
            t1._root._parent = node 
            node._left = t1._root 
            t1._root = None  # ??? 
            t1._size = 0
        if not t2.is_empty():
            t2._root._parent = node
            node._right = t2._root 
            t2._root = None
            t2._size = 0 

****Reinforcement Problems*** 